## Import torch and model

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

In [3]:
import sys
sys.path.append("../common")

from model_generator import ModelGenerator
from net import Net

## Set hyper params

In [4]:
batch_size = 128
num_epoch  = 200

lr = 0.1
gamma = 0.2             # learning rate decay
weight_decay = 0.0005

## for SGD
opt_momentum = 0.9
opt_nesterov = True

dropout_on = False
batchnorm_on = True 

scheduler_step_size = [60, 120, 160]

pretrained_model       = './cifar100_wrn_28_10_pretrained.pth'

## Load dataset

In [5]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))])

transform_train = transforms.Compose(
    [transforms.RandomHorizontalFlip(),
     transforms.RandomCrop(32, 4),
     transforms.ToTensor(),
     transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified


## WideResNet-28-10 model

In [6]:
model_gen = ModelGenerator(dropout = dropout_on, batchnorm = batchnorm_on)

model_gen.CifarWrnConfig(k = 10, num_layers = 28, cifar = 100)
model = model_gen.GetCifarWrn()

net = Net(model)

net.Gpu()

In [7]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(net.GetTotalParams(), lr=lr, weight_decay=weight_decay, momentum=opt_momentum, nesterov=opt_nesterov )

scheduler = lr_scheduler.MultiStepLR(optimizer, milestones = scheduler_step_size, gamma = gamma)

In [8]:
for epoch in range(num_epoch):  # loop over the dataset multiple times

    running_loss = 0.0
    scheduler.step()
    net.TrainMode()
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        inputs = Variable(inputs.cuda())
        labels = Variable(labels.cuda())

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        #scheduler.step()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss = (running_loss * i + loss.cpu().data.numpy()) / (i+1)

    correct = 0
    total = 0
    net.TestMode()
    for data in testloader:
        images, labels = data
        outputs = net(Variable(images.cuda()))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.cuda()).sum()
        
    print('%d epoch end, loss: %3.6f, Test Acc: %4.2f %%' %(epoch + 1, running_loss, 100 * correct / total))
    
print('\nTraining is finished!')

1 epoch end, loss: 3.961047, Test Acc: 14.96 %
2 epoch end, loss: 3.094576, Test Acc: 23.22 %
3 epoch end, loss: 2.447427, Test Acc: 36.80 %
4 epoch end, loss: 2.031867, Test Acc: 43.53 %
5 epoch end, loss: 1.778436, Test Acc: 45.29 %
6 epoch end, loss: 1.620624, Test Acc: 48.12 %
7 epoch end, loss: 1.499361, Test Acc: 47.26 %
8 epoch end, loss: 1.410976, Test Acc: 51.99 %
9 epoch end, loss: 1.334886, Test Acc: 52.16 %
10 epoch end, loss: 1.279441, Test Acc: 49.65 %
11 epoch end, loss: 1.228564, Test Acc: 53.72 %
12 epoch end, loss: 1.189186, Test Acc: 54.58 %
13 epoch end, loss: 1.150391, Test Acc: 53.02 %
14 epoch end, loss: 1.118233, Test Acc: 53.97 %
15 epoch end, loss: 1.099350, Test Acc: 58.02 %
16 epoch end, loss: 1.070504, Test Acc: 56.66 %
17 epoch end, loss: 1.045134, Test Acc: 54.01 %
18 epoch end, loss: 1.025262, Test Acc: 57.26 %
19 epoch end, loss: 1.014552, Test Acc: 55.36 %
20 epoch end, loss: 0.997343, Test Acc: 56.56 %
21 epoch end, loss: 0.979964, Test Acc: 59.24 %
2

171 epoch end, loss: 0.006729, Test Acc: 81.10 %
172 epoch end, loss: 0.006683, Test Acc: 80.99 %
173 epoch end, loss: 0.006628, Test Acc: 80.94 %
174 epoch end, loss: 0.006631, Test Acc: 80.91 %
175 epoch end, loss: 0.006581, Test Acc: 80.96 %
176 epoch end, loss: 0.006788, Test Acc: 81.02 %
177 epoch end, loss: 0.006689, Test Acc: 80.88 %
178 epoch end, loss: 0.006654, Test Acc: 81.03 %
179 epoch end, loss: 0.006725, Test Acc: 80.89 %
180 epoch end, loss: 0.006649, Test Acc: 80.92 %
181 epoch end, loss: 0.006732, Test Acc: 81.00 %
182 epoch end, loss: 0.006747, Test Acc: 80.85 %
183 epoch end, loss: 0.006880, Test Acc: 81.08 %
184 epoch end, loss: 0.006706, Test Acc: 81.07 %
185 epoch end, loss: 0.006655, Test Acc: 81.04 %
186 epoch end, loss: 0.006697, Test Acc: 81.02 %
187 epoch end, loss: 0.006967, Test Acc: 81.14 %
188 epoch end, loss: 0.006713, Test Acc: 81.22 %
189 epoch end, loss: 0.006792, Test Acc: 81.23 %
190 epoch end, loss: 0.006696, Test Acc: 81.02 %
191 epoch end, loss:

In [9]:
correct = 0
total = 0
net.TestMode()
for data in testloader:
    images, labels = data
    outputs = net(Variable(images.cuda()))
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()

print('Accuracy of the network on the 10000 test images: %4.2f %%' % (100 * correct / total))

Accuracy of the network on the 10000 test images: 81.19 %


In [10]:
torch.save(net.GetStateDict(), pretrained_model)